<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#⭐-STAR-⭐" data-toc-modified-id="⭐-STAR-⭐-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>⭐ STAR ⭐</a></span><ul class="toc-item"><li><span><a href="#🔎-Situation-🔍" data-toc-modified-id="🔎-Situation-🔍-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>🔎 Situation 🔍</a></span></li><li><span><a href="#⚠️-Threat-⚠️" data-toc-modified-id="⚠️-Threat-⚠️-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>⚠️ Threat ⚠️</a></span></li><li><span><a href="#⚒️-Action-⚒️" data-toc-modified-id="⚒️-Action-⚒️-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>⚒️ Action ⚒️</a></span></li><li><span><a href="#📋-Result-📋" data-toc-modified-id="📋-Result-📋-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>📋 Result 📋</a></span></li></ul></li><li><span><a href="#📦-Imports" data-toc-modified-id="📦-Imports-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>📦 Imports</a></span></li><li><span><a href="#Read-Data" data-toc-modified-id="Read-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read Data</a></span></li><li><span><a href="#🧼-Cleaning" data-toc-modified-id="🧼-Cleaning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>🧼 Cleaning</a></span></li><li><span><a href="#🔨-Workflow-1:-Keep-All-Row-Labels" data-toc-modified-id="🔨-Workflow-1:-Keep-All-Row-Labels-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>🔨 Workflow 1: Keep All Row Labels</a></span><ul class="toc-item"><li><span><a href="#Creating-Summary-for-Posting" data-toc-modified-id="Creating-Summary-for-Posting-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Creating Summary for Posting</a></span></li></ul></li><li><span><a href="#Summarizing-the-Summary" data-toc-modified-id="Summarizing-the-Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summarizing the Summary</a></span></li><li><span><a href="#ƒ:-Workflow" data-toc-modified-id="ƒ:-Workflow-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>ƒ: Workflow</a></span></li><li><span><a href="#So-Far..." data-toc-modified-id="So-Far...-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>So Far...</a></span></li><li><span><a href="#3.15.22" data-toc-modified-id="3.15.22-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>3.15.22</a></span></li><li><span><a href="#Generating-Alerts-for-RC:-Dupes-and-A&amp;G" data-toc-modified-id="Generating-Alerts-for-RC:-Dupes-and-A&amp;G-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Generating Alerts for RC: Dupes and A&amp;G</a></span><ul class="toc-item"><li><span><a href="#Using-Python-Logger" data-toc-modified-id="Using-Python-Logger-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Using Python Logger</a></span></li></ul></li><li><span><a href="#Checkpoint---4.19.22" data-toc-modified-id="Checkpoint---4.19.22-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Checkpoint - 4.19.22</a></span></li><li><span><a href="#Final-Script" data-toc-modified-id="Final-Script-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Final Script</a></span></li><li><span><a href="#⚠️-Old-Code---Raises-Error" data-toc-modified-id="⚠️-Old-Code---Raises-Error-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>⚠️ Old Code - Raises Error</a></span><ul class="toc-item"><li><span><a href="#✔️-FIXED-ERROR-✔️" data-toc-modified-id="✔️-FIXED-ERROR-✔️-14.1"><span class="toc-item-num">14.1&nbsp;&nbsp;</span>✔️ FIXED ERROR ✔️</a></span></li><li><span><a href="#⚠️-⚒️-TODO:-⚒️-⚠️" data-toc-modified-id="⚠️-⚒️-TODO:-⚒️-⚠️-14.2"><span class="toc-item-num">14.2&nbsp;&nbsp;</span>⚠️ ⚒️ TODO: ⚒️ ⚠️</a></span></li><li><span><a href="#Workflow-1:-Dropping-Unused-Labels" data-toc-modified-id="Workflow-1:-Dropping-Unused-Labels-14.3"><span class="toc-item-num">14.3&nbsp;&nbsp;</span>Workflow 1: Dropping Unused Labels</a></span></li><li><span><a href="#⚠️-CONDENSE-W/-UPPER-WF-⚠️" data-toc-modified-id="⚠️-CONDENSE-W/-UPPER-WF-⚠️-14.4"><span class="toc-item-num">14.4&nbsp;&nbsp;</span>⚠️ CONDENSE W/ UPPER WF ⚠️</a></span></li></ul></li></ul></div>

# Intro

# ⭐ STAR ⭐

## 🔎 Situation 🔍

---

* NA uses DRR summary reports to post revenue in LS
* I use report packet to review for DRR

---

## ⚠️ Threat ⚠️

---

* **Pro:** clearly provide total sum amounts for audit to post directly

* **Cons:**
    * "Food" = "Food" + "Other"
    * Discounts are ambiguous
        * FBC vs. Assoc vs. others
    * *LOTS* of unused rows/labels create visual confusion
    * Not always clear to which XAC to post revenue/tax/tips
    * Unusual/new/unused categories are not always accounted for

All issues result in manual review of all transactions during the Daily reporting process.

---
    

## ⚒️ Action ⚒️

---

**Create workflow to:**

* *Read existing DD reports*


* *Clean the report to isolate relevant details*


* *Report any irregularities:*
    * Any discounts or charges?
    * Any new/non-standard MOPs?
    * Report exact amounts to post to each LS XAC
        * See below

---

---

**End Goal:**

---

> **Post these amounts in LightSpeed:**

<!-- | |Dept | Rain 903 | IRD  | CC | Bar| 
| - | - | - | - | - | - |
|**Sub-Dept** | |  |  |  |
|Food | 1.00 | 1.00 | 1.00 | 1.00 |
|Liquor | 2.00 | 2.00 | 2.00 | 2.00 
|Tax | 3.00 | 3.00 | 3.00 | 3.00 
|Gratuity | 4.00 | 4.00 | 4.00 | 4.00  -->

|Dept| Food | Liquor | Tax  | Gratuity | 
| -  | -    | -      | -    | -        |
| DR | 1.00 | 1.00   | 1.00 | 1.00     |
| RS | 4.00 | 4.00   | 4.00 | 4.00     |
| CC | 1.00 | 1.00   | 1.00 | 1.00     |
| LB | 4.00 | 4.00   | 4.00 | 4.00     |

---

 >**Investigate these amounts:**
 
 | Item            | Amount |
 | -               | - |
 | Discount        | 1.00 |
 | Charge          | 1.00 |
 | $2 Tip - Coupon | 1.00 |

---

## 📋 Result 📋

# 📦 Imports

---

I will **use standard data reading/wrangling/EDA packages,** such as Numpy, Pandas, Matplotlib, and Seaborn, to prepare and review my data prior to generating the final report.

Additionally, I will **import my own personal EDA module,** containing a selection of hand-written functions to make the EDA process a bit easier.

Finally, I will **create two particular variables to help determine the source data:**
1. A Boolean variable controlling whether to use the reference files included in this repo (versus live/new reports).
2. A filepath as a string datatype to import the data.

---

In [ ]:
## Importing Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from datetime import datetime, timedelta

import logging

import bmcds.eda as eda

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', 
                    datefmt='%d-%b-%y %H:%M:%S',filename='logs.log',filemode='a')

## Adapted from http://docs.python.org/howto/logging-cookbook.html#logging-cookbook

# Create logger and set level
logger = logging.getLogger('Report_Logs')
logger.setLevel(logging.INFO)

# Create log file
fh = logging.FileHandler('logs.log')
fh.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)

#  Read Data

In [ ]:
## Specify file path

file_path = './data/02_25_2022_pc.xls'

In [ ]:
# ## Read specific columns from Excel file

# df = pd.read_excel(file_path, skiprows = 3, header = None,
#                    usecols = 'A, B, D, F, H', 
#                    names = ['Label', 'Lounge', 'Rain 903', 'In-Room Dining',
#                             'Coffee Corner'])
# df

In [ ]:
## Read all columns from file (incl. empty columns)

df = pd.read_excel(file_path, skiprows = 2)
df

# 🧼 Cleaning

---

I imported the data into a dataframe and immediately see that I will need to fix the alignment of my column names.

Additionally, there are several "Unnamed" columns (representing empty, unlabeled columns generated as part of the report). Knowing those columns are empty, I will drop those as well.

---

In [ ]:
## Shifting label names one to the right and drop extra column

df.columns = ['Label', *df.columns[:-1]]
df

In [ ]:
## Creating list of columns to keep/drop

col_labels = ['Label', 'Bar', 'Dining Room', 'Room Service', 'Starbucks']
drop_cols = []

for col in df.columns:
    if col not in col_labels:
        drop_cols.append(col)
        
drop_cols

In [ ]:
## Dropping columns not representing an outlet

df = df.drop(columns = drop_cols)
df

---

Now that I re-labeled the columns to match the data, I see there are blank rows between each row, similar to the blank columns experienced above.

Knowing that the reports do not have any rows that are either missing values or completely blank, I will drop rows with any blank values.

---

In [ ]:
## How many rows have missing values, and how many do they contain?

df.isna().sum(axis=1).value_counts()
# df.isna().sum(axis=0)

In [ ]:
## Displaying only rows without any missing values

df[df.isna().sum(axis=1) == 0]

In [ ]:
## Displaying only rows missing any values

# df[df.isna().sum(axis=1) > 0]
df[df.isna().sum(axis=1) > 0].value_counts(dropna=False)

---

The simple filtering above confirms my assumption that rows missing any number of missing values do not contain any relevant information. In the normal printout, these rows would be blank spaces between lines.

Additionally, I noticed there are a few rows that are "blank;" they contain some sort of data, but they do not provide meaningful information.

I will filter out rows with missing values and blank rows.

---

In [ ]:
## Checking size of dataframe prior to dropping rows
df.shape

In [ ]:
## Checking for blank rows
df['Label'].value_counts()

In [ ]:
## Reviewing rows containing a blank space for a label
df[df.loc[:, "Label"].isin([' '])]

In [ ]:
## Filter out those rows with missing/blank values

df_cleaned = df[df.isna().sum(axis=1) == 0]
df_cleaned = df_cleaned[~df_cleaned.loc[:, "Label"].isin([' '])]
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned

In [ ]:
## Checking number of rows post-drop

df_cleaned.shape

In [ ]:
## Inspecting datatype and remaining data
df_cleaned.info()

---

**Review Current Status**

At this point, I completed the following:

- [X] Imported specified report data
- [X] Removed blank columns and rows
- [X] Removed any rows missing values

At this point, I condensed the report and removed any extra spacing.

**Now I need to decide which rows contain relevant data to keep and which rows contain extra data that we don't use.**

---

**Keeping vs. Dropping**

My original intention for this notebook/eventual script was to create a very minimalistic summary of the relevant day-to-day data, cutting out the extra stuff that we ignore.
- This would provide me with the data specifically used during our night audit processes and remove any confusing extra details.

However, I realized that approach would cause problems if/when we introduce new payment methods or would have any rare situations where relevant data would come through to the reports. 
- Missing such data could result in problems creating the daily reports or worse - obscure potential problems/irregularities.

---

**Final Decision:**

I will focus my code on keeping all row labels while maintaining a focus on the most important day-to-day details.
- To address the concern about irregularities, I will create warnings to highlight such data.

---

# 🔨 Workflow 1: Keep All Row Labels

---

To create my ideal report, **I will focus on specific rows that I use most often for my reporting.** 

I will split the remaining rows into two groups:
1. Regularly-occurring (but irrelevant) data
    - This includes subtotals and most of the rows at the end
    - These rows are irrelevant for the day-to-day reports


2. Infrequent but highly valuable data
    - This group would also act as a catch-all for any new rows, such as when we create a new payment method.

**I will sub-divide the dataframe into the three new dataframes, one for each group.**

- Both the "regular" data and "infrequent-but-valuable" dataframes will be included on the first sheet of the final Excel workbook
- The extra dataframe will be saved to the second page.

---

In [ ]:
## Create new DF

df_all_cols = df_cleaned.copy().T
df_all_cols

In [ ]:
## Re-labeling columns based on "Label" row
df_all_cols.columns = df_all_cols.iloc[0]
df_all_cols = df_all_cols.drop(index = 'Label')
df_all_cols

In [ ]:
## Specifying daily-use labels

daily_rows = ['Food', 'Beverage', 'Other', 'Discount', 'MD Food 6%',
              'MD Liq 9%', 'Tip collected', "ALL A&G CHRG", 'Room Charge',
              'American Express', 'Cash', 'Discover Card', 'MasterCard', 'Visa']

df_daily = df_all_cols[daily_rows]
df_daily

In [ ]:
## Converting data to "float"
for col in df_daily.columns:
    df_daily.loc[:,col] = pd.to_numeric(df_daily.loc[:,col], downcast = 'float')
    
df_daily

## Creating Summary for Posting

---

**My main goal is to create a simple, easy-to-use report that I can roll-out quickly and easily.** Ideally, the summary will be intuitive, clear, and allow for the night auditor to post the charges quickly and easily.

I know that there are only a select few columns that we reference for posting charges, so I will start my summary by creating a small dataframe summarizing the usual charges.

---

In [ ]:
## Summarize food charges by outlet
daily_food = df_daily['Food'] + df_daily['Other'] - df_daily['Discount']
daily_food

In [ ]:
## Summarize tax charges by outlet
daily_tax = df_daily['MD Food 6%']+df_daily['MD Liq 9%']
daily_tax

In [ ]:
## Summarize all charges by outlet
summary = pd.concat([daily_food, df_daily['Beverage'], daily_tax, 
           df_daily['Tip collected']], axis=1)
summary

In [ ]:
## Creating new labels

col_labels = {0: "Food", 1: "Tax", 'Tip collected': 'Gratuity'}

row_labels = {'Bar': "Lobby Bar", "Dining Room": 'Rain 903', 
              'Room Service': 'In-Room Dining', 'Starbucks': 'Coffee Corner'}

col_labels, row_labels

In [ ]:
## Renaming columns
summary = summary.rename(columns = col_labels, index = row_labels)
summary

In [ ]:
## Rounding values - preventing long numbers
summary = summary.applymap(lambda x: round(x, 2))
summary

In [ ]:
# summary.to_excel('Test_Summary_Workbook.xlsx')

# Summarizing the Summary

---

First summary working - need to expand

* Repeat process per MOP
    * A&G
    * RC
    * CA
    * AX
    * DI
    * VI/MC
    
* Save results to different sheets?

---

# ƒ: Workflow

---

 - [ ] Create workflow function to reuse w/ each report
 - [ ] Test specific MOP reports
 

In [ ]:
def cleaning_mops(file_path, new_file = None):

    df = pd.read_excel(file_path, skiprows = 2)

    df.columns = ['Label', *df.columns[:-1]]

    col_labels = ['Label', 'Bar', 'Dining Room', 'Room Service', 'Starbucks']
    drop_cols = []

    for col in df.columns:
        if col not in col_labels:
            drop_cols.append(col)

    df = df.drop(columns = drop_cols)

    df_cleaned = df[df.isna().sum(axis=1) == 0]
    df_cleaned = df_cleaned[~df_cleaned.loc[:, "Label"].isin([' '])]
    df_cleaned = df_cleaned.reset_index(drop=True)

    df_all_cols = df_cleaned.T

    df_all_cols.columns = df_all_cols.iloc[0]
    df_all_cols = df_all_cols.drop(index = 'Label')

    daily_rows = ['Food', 'Beverage', 'Other', 'Discount', 'MD Food 6%',
              'MD Liq 9%', 'Tip collected', "ALL A&G CHRG", 'Room Charge',
              'American Express', 'Cash', 'Discover Card', 'MasterCard', 'Visa']

    df_daily = df_all_cols.loc[:,daily_rows]

    for col in df_daily.columns:
        df_daily.loc[:,col] = pd.to_numeric(df_daily.loc[:,col], downcast = 'float')

    daily_food = df_daily.loc[:,'Food'] + df_daily.loc[:,'Other'] - df_daily.loc[:,'Discount']

    daily_tax = df_daily.loc[:,'MD Food 6%'] + df_daily.loc[:,'MD Liq 9%']

    summary = pd.concat([daily_food, df_daily.loc[:,'Beverage'], daily_tax, 
           df_daily.loc[:,'Tip collected']], axis=1)
    
    col_labels = {0: "Food", 1: "Tax", 'Tip collected': 'Gratuity'}

    row_labels = {'Bar': "Lobby Bar", "Dining Room": 'Rain 903', 
              'Room Service': 'In-Room Dining', 'Starbucks': 'Coffee Corner'}

    summary = summary.rename(columns = col_labels, index = row_labels)

    summary = summary.applymap(lambda x: round(x, 2))

#     summary.to_excel(new_file)
    return summary

In [ ]:
test_df = cleaning_mops(file_path = './data/02_25_2022_mop_det_VIMC.xls')
test_df

In [ ]:
test_df['MoP'] ='VI/MC'
test_df

In [ ]:
test_df = test_df.reset_index()
test_df = test_df.set_index(['MoP', 'index'])
test_df

In [ ]:
mop = 'AG'
file_path = f'./data/02_25_2022_mop_det_{mop}.xls'
results = cleaning_mops(file_path)
results.loc[:,'MoP'] = mop.upper()

results

In [ ]:
list_mops = ['ag', 'rc', 'ax', 'ca', 'di', 'vimc']

results = pd.DataFrame()

for mop in list_mops:
    file_path = f'./data/02_25_2022_mop_det_{mop}.xls'
    results = cleaning_mops(file_path)
    results.loc[:,'MoP'] = mop.upper()

In [ ]:
results.keys()

In [ ]:
results

# So Far...

---

**√: MVP**

- [x] Created function to clean, organize data from summary reports
 - [x] Tested results by looping through list of files


**TD: MVP**

 - [ ] Add additional column identifying MOP
 - [ ] Create multi-index w/ MOP, PC
 - [ ] Concat DFs
 - [ ] Save results to XL
 - [ ] Convert to script
 
 
**TD: AAB**

 - [ ] Create filters to include extra details
     * *Ex: Discounts, charges, any new MOPs, payments other than listed MOPs*

---

# 3.15.22

Goal: MVP up & running

 - [ ] Check status
 - [ ] Add/create multi-index for MOP
 - [ ] Create SINGLE results XL

In [ ]:
def cleaning_mops_test(file_path, mop_name = None, new_file_name = None):
    
    ## Read file
    df = pd.read_excel(file_path, skiprows = 2)
    
    ## Shift label names one to the right and drop extra column
    df.columns = ['Label', *df.columns[:-1]]
    
    ## Specify features to keep
    col_labels = ['Label', 'Bar', 'Dining Room', 'Room Service', 'Starbucks']
    drop_cols = []
    
    ## Create list of column names to drop
    for col in df.columns:
        if col not in col_labels:
            drop_cols.append(col)
    
    ## Drop columns
    df = df.drop(columns = drop_cols)
    
    ## Drop rows with NaNs and/or blank labels
    df_cleaned = df[df.isna().sum(axis=1) == 0]
    df_cleaned = df_cleaned[~df_cleaned.loc[:, "Label"].isin([' '])]
    df_cleaned = df_cleaned.reset_index(drop=True)
    
    ## Transpose for easy reading
    df_all_cols = df_cleaned.T
    
    ## Relabel based on values in first row
    df_all_cols.columns = df_all_cols.iloc[0]
    df_all_cols = df_all_cols.drop(index = 'Label')

    daily_rows = ['Food', 'Beverage', 'Other', 'Discount', 'MD Food 6%',
              'MD Liq 9%', 'Tip collected', "ALL A&G CHRG", 'Room Charge',
              'American Express', 'Cash', 'Discover Card', 'MasterCard', 'Visa']
    
    ## Slicing out specific labels of interest
    df_daily = df_all_cols.loc[:, daily_rows]
    
    ## Recast datatypes
    for col in df_daily.columns:
        df_daily.loc[:,col] = pd.to_numeric(df_daily.loc[:,col], downcast = 'float')

    ## Generate totals for each outlet
    daily_food = df_daily['Food'] + df_daily['Other'] - df_daily['Discount']

    daily_tax = df_daily['MD Food 6%'] + df_daily['MD Liq 9%']

    summary = pd.concat([daily_food, df_daily.loc[:,'Beverage'], daily_tax,
                         df_daily.loc[:,'Tip collected']], axis=1)
    
    ## Relabel columns/rows
    col_labels = {0: "Food", 1: "Tax", 'Tip collected': 'Gratuity'}
    row_labels = {'Bar': "Lobby Bar", "Dining Room": 'Rain 903', 
              'Room Service': 'In-Room Dining', 'Starbucks': 'Coffee Corner'}
        
    summary = summary.rename(columns = col_labels, index = row_labels)
    
    ## Round to prevent unnecessary decimals
    summary = summary.applymap(lambda x: round(x, 2))
    
    ## Generate new index to group MOPs
    if mop_name != None:
        summary.loc[:,'Payment'] = mop_name.upper()
        summary = summary.reset_index()
        summary = summary.rename(columns = {'index':'Outlet'})
        summary = summary.set_index(['Payment', 'Outlet'])
        
#     summary.to_excel(new_file)
    return summary

In [ ]:
## For a specified MOP, pull specific file and run cleaning function

mop = 'vimc'

file_path = f'./data/02_25_2022_mop_det_{mop}.xls'

cleaning_mops_test(file_path=file_path, mop_name=mop)

In [ ]:
## Run all MOPs through function

list_mops = ['ag', 'rc', 'ax', 'ca', 'di', 'vimc']

new_results = []

for mop in list_mops:
    file_path = f'./data/02_25_2022_mop_det_{mop}.xls'
    new_results.append(cleaning_mops_test(file_path=file_path, mop_name=mop))
    
new_results_df = pd.concat(new_results)
new_results_df

In [ ]:
# new_results_df.to_excel('./data/02_25_2022_summary.xlsx')

# Generating Alerts for RC: Dupes and A&G

* Two possible errors for Room Charge settlements:
    * Duplicate checks
    * Charged to A&G
* Need report to check guest names to ID and warn of these postings

In [ ]:
ra_rc_df = pd.read_excel(io = './data/april_2022_rc_receipts.xls', header = None, skiprows = 1,
                        names = ['Table #', 'Check #', 'Server #', 'Server Name', 'Cashier #',
                                 'Cashier Name','MoP', 'Profit Center', 'Blank1', 'Blank2',
                                 'Tip', 'Total Receipt', 'Guest Name/Room', 'Date', 'Time'])
ra_rc_df

In [ ]:
## Dropping total rows at end of report
ra_rc_df = ra_rc_df[:-3]
ra_rc_df

In [ ]:
bool_idx = ra_rc_df.loc[:,'Guest Name/Room'].str.startswith(('A&G', 'Dup'))
bool_idx

In [ ]:
## Slicing out checks that meet the criteria
ra_rc_df[bool_idx]

In [ ]:
chk_num = list(ra_rc_df[bool_idx]['Check #'].values)
chk_num

## Using Python Logger

References:
* [Real Python](https://realpython.com/python-logging/)
* [Python Docs](https://docs.python.org/3/howto/logging.html#logging-basic-tutorial)

In [ ]:
print(f'Review check numbers {chk_num} for duplicate check/direct A&G postings!')

In [ ]:
if len(chk_num) >0:
    logger.warning(f'Review checks for possible errors: {chk_num}')

# Checkpoint - 4.19.22

---

**Current Status:**

* Read Digital Dining report data *only for Excel files*
    * AAB: read DBF files directly
    * AAB: review sales type for discounts, subtract from proper sales type
        * Currently subtracted from food generally
* Clean the data to generate summaries about food/bev/tax/gratuity amounts
    * MVP: confirm process works for all MOPs
* Log warnings about any "Duplicate Check" or direct A&G postings

---

**Next Steps:**

1. Test on most recent files
2. Artificially generate errors (then cancel before NA)
3. Condense into script to run on server
4. Test script
5. Schedule script to run daily

---

# Final Script

In [ ]:
def workflow(data_filepath, mop_name = None):#, summary_file_name, receipts_filepath, log_filepath):
    """Processes entire workflow to generate summary data for Digital Dining reports.

    Reads files; cleans; summarizes by MOP; generates log files to confirm any issues.

    Args:
        data_filepath (str): Path to data file (.xls/.xlsx files only)
        summary_file_name (str): Name for results file
        receipts_filepath (str): Receipts Audit report file
        log_filepath (str): Name for log file.
    """

#     logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', 
#                         datefmt='%d-%b-%y %H:%M:%S',filename=log_filepath,
#                         encoding='utf-8',  filemode='a')

#     # Create logger and set level
#     logger = logging.getLogger('Report_Log')
#     logger.setLevel(logging.INFO)

#     # Create log file handler
#     fh = logging.FileHandler(log_filepath)
#     fh.setLevel(logging.INFO)

#     formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#     fh.setFormatter(formatter)

#     logger.addHandler(fh)



    ## Read all columns from file (incl. empty columns)
    df = pd.read_excel(data_filepath,  skiprows = 2)

    ## Shifting label names one to the right and drop extra column
    df.columns = ['Label', *df.columns[:-1]]

    ## Creating list of columns to keep/drop
    col_labels = ['Label', 'Bar', 'Dining Room', 'Room Service', 'Starbucks']
    drop_cols = []

    for col in df.columns:
        if col not in col_labels:
            drop_cols.append(col)

    ## Dropping columns not representing an outlet
    df = df.drop(columns = drop_cols)

    df_cleaned = df[df.isna().sum(axis=1) == 0]
    df_cleaned = df_cleaned[~df_cleaned.loc[:, "Label"].isin([' '])]
    df_cleaned = df_cleaned.reset_index(drop=True)

    df_all_cols = df_cleaned.copy().T
    df_all_cols

    ## Re-labeling columns based on "Label" row
    df_all_cols.columns = df_all_cols.iloc[0]
    df_all_cols = df_all_cols.drop(index = 'Label')

    daily_rows = ['Food', 'Beverage', 'Other', 'Discount', 'MD Food 6%',
                'MD Liq 9%', 'Tip collected', "ALL A&G CHRG", 'Room Charge',
                'American Express', 'Cash', 'Discover Card', 'MasterCard', 'Visa']

    df_daily = df_all_cols[daily_rows]

    ## Converting data to "float"
#     for col in df_daily.columns:
#         df_daily.loc[:,col] = pd.to_numeric(df_daily.loc[:,col], downcast = 'float')
    df_daily = df_daily.applymap(lambda x: float(x))

    ## Summarize food charges by outlet
    daily_food = df_daily['Food'] + df_daily['Other'] - df_daily['Discount']

    ## Summarize tax charges by outlet
    daily_tax = df_daily['MD Food 6%']+df_daily['MD Liq 9%']

    ## Summarize all charges by outlet
    summary = pd.concat([daily_food, df_daily['Beverage'], daily_tax, 
            df_daily['Tip collected']], axis=1)

    ## Creating new labels
    col_labels = {0: "Food", 1: "Tax", 'Tip collected': 'Gratuity'}
    row_labels = {'Bar': "Lobby Bar", "Dining Room": 'Rain 903', 
                'Room Service': 'In-Room Dining', 'Starbucks': 'Coffee Corner'}

    ## Renaming columns
    summary = summary.rename(columns = col_labels, index = row_labels)

    ## Rounding values - preventing long numbers
    summary = summary.applymap(lambda x: round(x, 2))
    
    ## Generate new index to group MOPs
    if mop_name != None:
        summary.loc[:,'Payment'] = mop_name.upper()
        summary = summary.reset_index()
        summary = summary.rename(columns = {'index':'Outlet'})
        summary = summary.set_index(['Payment', 'Outlet'])

#     # summary.to_excel(summary_file_name, engine = 'openpyxl')

#     ra_rc_df = pd.read_excel(io = receipts_filepath, header = None, skiprows = 1,
#                         names = ['Table #', 'Check #', 'Server #', 'Server Name', 'Cashier #',
#                                 'Cashier Name','MoP', 'Profit Center', 'Blank1', 'Blank2',
#                                 'Tip', 'Total Receipt', 'Guest Name/Room', 'Date', 'Time'])
    
#     ## Dropping total rows at end of report
#     ra_rc_df = ra_rc_df[:-3]

#     bool_idx = ra_rc_df.loc[:,'Guest Name/Room'].str.startswith(('A&G', 'Dup'))

#     chk_num = list(ra_rc_df[bool_idx]['Check #'].values)

#     if len(chk_num) >0:
#         logger.warning(f'Review checks for possible errors: {chk_num}')
#     else:
#         logger.info(f'There are no checks to review.')

    return summary

In [ ]:
workflow(data_filepath = './data/02_25_2022_pc.xls')

In [ ]:
def loop_workflow():#, summary_file_name, receipts_filepath, log_filepath):

    
    list_mops = ['ag', 'rc', 'ax', 'ca', 'di', 'vimc']
    new_results = []
    for mop in list_mops:
        data_filepath = f'./data/02_25_2022_mop_det_{mop}.xls'
        summary_file_name = f'results_for_{mop}.xlsx'
        new_results.append(workflow(data_filepath, mop_name = mop))#, summary_file_name, receipts_filepath, log_filepath))
        
    new_results_df = pd.concat(new_results)

    return new_results_df

In [ ]:
loop_workflow()

In [ ]:
from functions import script
# from functions.loop_script import loop_workflow

In [ ]:
workflow(data_filepath ='./data/02_25_2022_mop_det_vimc.xls', summary_file_name = 'test_workflow_results.xlsx',
         receipts_filepath ='./data/april_2022_rc_receipts.xls', log_filepath = 'test_workflow_log.log')

In [ ]:
# def loop_workflow(data_filepath, summary_file_name, receipts_filepath, log_filepath):

    
#     list_mops = ['ag', 'rc', 'ax', 'ca', 'di', 'vimc']
#     new_results = []
#     for mop in list_mops:
#         file_path = f'./data/02_25_2022_mop_det_{mop}.xls'
#         summary_file_name = f'results_for_{mop}.xlsx'
#         new_results.append(workflow(data_filepath, summary_file_name, receipts_filepath, log_filepath))
        
#     new_results_df = pd.concat(new_results)

#     return new_results_df

In [ ]:
loop_workflow(summary_file_name = 'test_workflow_results.xlsx',
              receipts_filepath ='./data/april_2022_rc_receipts.xls',
              log_filepath = 'test_workflow_log.log')

In [ ]:
# receipts_filepath ='./data/april_2022_rc_receipts.xls'
# log_filepath = 'test_workflow_log.log'

# list_mops = ['ag', 'rc', 'ax', 'ca', 'di', 'vimc']

# new_results = []

# for mop in list_mops:
#     file_path = f'./data/02_25_2022_mop_det_{mop}.xls'
#     summary_file_name = f'results_for_{mop}.xlsx'
#     new_results.append(workflow(data_filepath = file_path, summary_file_name = summary_file_name,
#                                 receipts_filepath = receipts_filepath, log_filepath = log_filepath))

# new_results_df = pd.concat(new_results)

# ⚠️ Old Code - Raises Error

In [ ]:
raise Exception ("End of Development - Resume here")

In [ ]:
## Review datatypes

df_no_drp.dtypes

In [ ]:
## Recast columns to "float" datatype & review

for col in df_no_drp.columns:
    df_no_drp[col] = pd.to_numeric(df_no_drp[col], downcast = 'float')

print(df_no_drp.dtypes)

display(df_no_drp)

In [ ]:
## Inspect results

df_no_drp[df_no_drp.columns[:-1]].T.describe().T

In [ ]:
# ## Create new total column

# df_no_drp['Total'] = df_no_drp.sum(axis=1)
# df_no_drp

In [ ]:
## Filter rows without any activity

## CURRENTLY UNUSED - FILTERING BELOW ##

# df_no_zero = df_no_drp[df_no_drp['Total'] > 0]
# df_no_zero

In [ ]:
## Identify all unique labels from total data

df_no_drp.index.unique()

In [ ]:
## Create list of frequently-used labels

freq_cats = ['Food', 'Beverage', 'Other','Discount', 'Charge','MD Food 6%',
             'MD Liq 9%','Tip collected','ALL A&G CHRG', 'Room Charge',
             'American Express', 'Cash', 'Discover Card', 'MasterCard',
             'Visa']
freq_cats

In [ ]:
## Filtering frequent labels from total data

df_no_drp.loc[freq_cats, :]

In [ ]:
## Creating lists to subset the data

cats_profit_centers = df_no_drp.columns[:-1]
cats_revenue = ['Food','Beverage','Other','Charge']
cats_disc = ['Discount']
cats_tax = ['MD Food 6%', 'MD Liq 9%']
cats_tips = ['Tip collected']
cats_mops = ['ALL A&G CHRG', 'American Express', 'Room Charge', 'Cash',
             'Discover Card', 'MasterCard', 'Visa']

In [ ]:
# bar_post_rev = {}

# for cat in cats_revenue:
# #     bar_post_rev[cat] = df_no_drp.loc[freq_cats,cat]
#     print(df_no_drp.loc[cat, 'Lounge'])

##   ✔️ FIXED ERROR ✔️

* Need to switch to individual MOP reports
    * PC report shows total sums
    * Need to break down by MOP for proper details

In [ ]:
## Pulling yesterday's files
## Useful for processing during Daily reporting, but not same-day
# yesterday = datetime.now() - timedelta(1)    

# yd_str = datetime.strftime(yesterday, '%m_%d_%Y')
# yd_str

In [ ]:
## Pulling today's files

today = datetime.now()   

td_str = datetime.strftime(today, '%m_%d_%Y')
td_str

In [ ]:
list_mops = ['ag', 'ax', 'ca', 'di', 'rc', 'vimc']

In [ ]:
def clean_data(file_path):
    """Work flow to import, clean, and eventually save the results as a new report.

    Args:
        file_path (string): file path for the original profit center report
    """    

    ## Read specific columns from Excel file

    df = pd.read_excel(file_path, skiprows = 2)

    ## Reassign label names: shifting to the right and dropping extra

    df.columns = ['Label', *df.columns[:-1]]
    ## Creating list of columns to keep/drop

    col_labels = ['Label', 'Bar', 'Dining Room', 'Room Service', 'Starbucks',
                'Page Total']

    drop_cols = []

    for col in df.columns:
        if col not in col_labels:
            drop_cols.append(col)
            
    ## Dropping columns not representing an outlet

    df = df.drop(columns = drop_cols)

    ## Filter out those rows with missing/blank values

    df_cleaned = df[df.isna().sum(axis=1) == 0]
    df_cleaned = df_cleaned[~df_cleaned.loc[:, "Label"].isin([' '])]
    df_cleaned = df_cleaned.reset_index(drop=True)

    ## Create new DF to allow for later reuse

    df_no_drp = df_cleaned.copy()

    ## Relabel and reset index with correct labels

    df_no_drp = df_no_drp.rename(df_no_drp["Label"])
    df_no_drp = df_no_drp.drop(columns = "Label")

    ## Recast columns to "float" datatype & review

    for col in df_no_drp.columns:
        df_no_drp[col] = pd.to_numeric(df_no_drp[col], downcast = 'float')
    
#     ## Create list of frequently-used labels

#     freq_cats = ['Food', 'Beverage', 'Other','Discount', 'Charge','MD Food 6%',
#              'MD Liq 9%','Tip collected','ALL A&G CHRG', 'Room Charge',
#              'American Express', 'Cash', 'Discover Card', 'MasterCard',
#              'Visa']

    return df_no_drp

In [ ]:
## Pulling all MOP reports and saving to dict

if use_ref_rpt == True:
    yd_str = '02_25_2022'
else:
    yd_str = datetime.strftime(yesterday, '%m_%d_%Y')

dict_reports = {}

for mop in list_mops:
    dict_reports[mop] = clean_data(f'./data/{yd_str}_mop_det_{mop}.xls')

In [ ]:
## Review resulting keys
dict_reports.keys()

In [ ]:
## Check number of columns in each report
for key in dict_reports:
    print(key, dict_reports[key].shape[1])

## ⚠️ ⚒️ TODO: ⚒️ ⚠️

1. Filter non-zero dataframe into common/uncommon categories
2. Create DF w/ LS XAC for normal use
3. Create DF w/ unusual cats w/ prompt to review

## Workflow 1: Dropping Unused Labels

---

**Dropping Labels**

> Currently, I ignore several sections of the report as they do not report any relevant details. However, **this may lead us to overlook any irregular data.**
>
> This work flow focused on the most commonly used section of the report, highlighting the details used for NA.
>
> While I prefer a more inclusive approach, I am keeping this work flow for reference.

---

In [ ]:
## Identify unique labels to determine which to drop

df_cleaned.loc[:, "Label"].unique()

In [ ]:
## Create list of labels to drop

drop_labels = ['Sales Subtotal', 'Total before tax', 'None', 'None', 
               'Tax Subtotal','Receipt Rounding','Total of check', ' ', 'Unadjusted receipts',
               'PaidIn - AR', 'PaidIn - Debitek','PaidIn - Gift Certificates',
               'No of Covers', 'No of Checks', 'Avg cover', 'Avg check',
               'Total Paid Outs', 'Server bank', 'Total tips',
               'Server credit card fees', 'Net tips', 'Net Cash',
               'Other receipts turned in', 'Server Drops', 'Turned in',
               'Restaurant credit card fees', 'Estimated deposit',
               'Beginning NRS Total', 'Total Receipts', 'Discounts',
               'Paid Ins','Gross Total', 'NRS Total']
drop_labels

In [ ]:
## Drop specified labels and save to new DF

df_new_idx =df_cleaned[~df_cleaned.loc[:, "Label"].isin(drop_labels)]
df_new_idx

In [ ]:
## Rename index with remaining labels

df_new_idx = df_new_idx.rename(df_cleaned["Label"])
df_new_idx = df_new_idx.drop(columns = "Label")
df_new_idx

## ⚠️ CONDENSE W/ UPPER WF ⚠️

In [ ]:
## Review remaining datatypes

df_new_idx.dtypes

In [ ]:
## Recast DT

for col in df_new_idx.columns:
    df_new_idx[col] = pd.to_numeric(df_new_idx[col], downcast = 'float')

df_new_idx.dtypes

In [ ]:
## Inspect results

df_new_idx.T.describe()

In [ ]:
## Create new total column

df_new_idx['Total'] = df_new_idx.sum(axis=1)
df_new_idx

In [ ]:
## Filter out zero sums

df_compressed = df_new_idx[df_new_idx['Total'] >0]
df_compressed

In [ ]:
df_compressed['Lounge']['Food']

In [ ]:
## Test print statements for logging purposes

for col in df_compressed.columns[:-1]:
    print(f"Post to {col} food: {df_compressed[col]['Food']+df_compressed[col]['Other']:.2f}")

In [ ]:
## Further testing for logging purposes

dept_list = ["Rain 903", "In Room Dining", "Coffee Corner", "Lobby Bar"]
sd_list = ["Food", "Liquor", "Tax", "Gratuity"]

for dept in dept_list:
    for sd in sd_list:
        print(dept+" - "+sd)